In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('jeju.csv',encoding='cp949')
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20


In [3]:
# 1. 주소 정보를 이용해서 위도,경도 값을 저장합니다. '위치' 컬럼에 (위도,경도) 저장
# 주소를 찾아서 저장하는 함수를 작성하여 처리  
# 함수 이름은 geo_coder(query) 리턴값 : (위도,경도)  만일 못찾았다면 '결과없음' 으로 리턴
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [16]:
def geo_coder(query):
    import requests
    REST_API_KEY = 'key'
    query = query.split('(')[0]
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?category_group_code=PO3&query={query}"
    headers = {'Authorization':f'KakaoAK {REST_API_KEY}'}
    res = requests.get(url,headers=headers)
    jsondata = res.json()
    # print(query)
    # print(jsondata)
    try:
        return jsondata['documents'][0]['y'],jsondata['documents'][0]['x']
    except:
        return '결과없음'

In [17]:
df['위치'] = df['주소'].apply(geo_coder)

In [18]:
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,위치,인구수
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,"(33.5150737361162, 126.526342790758)",3146


In [19]:
# 2. '위치' 컬럼의 값을 Marker 위치값으로, tooltip으로 읍면동을 사용하여 지도에 표시(클리스터이용)
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [20]:
import folium
from folium.plugins import MarkerCluster

In [21]:
def display(loc,map):
    # print(loc[0],loc[1])
    if loc[0] != '결과없음':
        folium.Marker(loc[0],tooltip=loc[1]).add_to(map)

In [22]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
df[['위치','읍면동']].apply(display,map=map_c,axis=1)
map

In [23]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
for index, row in df.iterrows():
    # print(index)
    # print(row['읍면동'],row['위치'])
    if row['위치'] != '결과없음':
        folium.Marker(row['위치'],tooltip=row['읍면동']).add_to(map_c)
map

In [24]:
# 3. '일반현황'컬럼의 인구수만 분리하여 '인구수'라는 int형 컬럼을 생성

In [25]:
df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.split().str.get(-1).str.replace('명','').astype(int)

0      3146
1     38400
2      7418
3     41428
4     14281
5      9431
6      8368
7     16701
8     10850
9     25559
10    25000
11     3046
12    28741
13    13729
14    39897
15    56223
16    17606
17     4104
18     2837
19    19925
20    26539
21    15080
22    20804
23     8745
24     2820
25     1718
26    16610
27    19151
28    13866
29    11867
30    11167
31     5207
32     2444
33     4003
34     3798
35     5475
36     4980
37    22225
38    10697
39    10344
40    11310
41    10684
42     3813
Name: 일반현황, dtype: int32

In [26]:
import re

df['인구수'] = df['일반현황'].apply(lambda x:int(re.findall('[0-9]{4,}',x)[-1]))

In [27]:
df.head(2)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,위치,인구수
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,"(33.5150737361162, 126.526342790758)",3146
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,결과없음,38400


In [29]:
# 4. 지역을 제주시는 "Jeju", 서귀포시는 "Seogwipo" 변경
df['키'] = df['지역'].replace(['제주시','서귀포시'],["Jeju","Seogwipo"]) 

In [30]:
df.sample(3)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,위치,인구수,키
11,제주시,봉개동,064-728-4742,제주특별자치도 제주시 봉개북1길 15,인구 3046명 / 면적47.2㎢ / 7통30반,지하 1층·지상2층 / 연면적 770㎡,총면적 390㎡ / 개소일 2000-10-05,2019-06-20,"(33.4917232671377, 126.594687244151)",3046,Jeju
29,서귀포시,안덕면,064-760-4320,제주특별자치도 서귀포시 안덕면 화순서서로 74,인구 11867명 ('18.12.31.기준) / 면적 105.58㎢ / 12개리 1...,지상 2층,총면적 155㎡ / 개소일 2000-11-14,2019-06-20,"(33.2576271446112, 126.330674173688)",11867,Seogwipo
26,서귀포시,대정읍,064-760-4012,제주특별자치도 서귀포시 대정읍 하모중앙로 20,인구 16610명 / 면적 78.53㎢ / 23개리 240개반,지상 2층·지하 1층 / 연면적 986.95㎡(본관),총면적 149㎡ / 개소일 2006-08-29,2019-06-20,"(33.22666983359228, 126.25181131648702)",16610,Seogwipo


In [35]:
# 5. 지역별 데이터 집계
df1 = df.groupby('키')['인구수'].sum()
df1= df1.to_frame().reset_index()

In [36]:
df1

,키,인구수
0,Jeju,462396
1,Seogwipo,167641


In [37]:
import folium,json

In [42]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
jsonfile = open('jeju-municipalities-geo.json','r')
jsondata = json.load(jsonfile)
# jsondata
folium.Choropleth(geo_data=jsondata,
                  data=df1,
                  columns=['키','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map

In [ ]:
# 6. 읍면동별로 인구수를 표시

In [44]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
jsonfile = open('jeju.json','r')
jsondata = json.load(jsonfile)
# jsondata
folium.Choropleth(geo_data=jsondata,
                  data=df,
                  columns=['읍면동','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map

In [52]:
jsonfile = open('HangJeongDong_ver20230101.geojson','r',encoding='utf8').read()
jsondata = json.loads(jsonfile)
# jsondata
jsondata_jeju = {'type': 'FeatureCollection'}
jsondata_pick = []
jsondata_dong = []
for idx in jsondata['features']:
    if idx['properties']['sidonm'] == "제주특별자치도":
        dong = idx['properties']['adm_nm'].split()[-1].strip()
        # print(dong)
        jsondata_dong.append(dong)
        idx['id']=dong
        jsondata_pick.append(idx)
jsondata_jeju['features']=jsondata_pick

In [54]:
# jsondata_jeju

In [55]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
# jsondata
folium.Choropleth(geo_data=jsondata_jeju,
                  data=df,
                  columns=['읍면동','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map